In [ ]:
#importando bibliotecas
import pandas as pd 
import csv, sqlite3
import os
import re

In [ ]:
con = sqlite3.connect("./censo_matricula.db") 
cur = con.cursor()

In [ ]:
cur.execute('DROP TABLE IF EXISTS matricula')
cols = re.sub(u'[^a-zA-Z0-9,_]', '', str(list(csv.reader(open('../input/censo-2020-parse/matricula_co-000.CSV', 'r'),delimiter=','))[0]))
cur.execute("CREATE TABLE IF NOT EXISTS matricula ("+cols+");")

In [ ]:
files = [] 
for diretorio, subpastas, arquivos in os.walk('../input/censo-2020-parse'):
    for arquivo in arquivos:
        files.append(os.path.join(os.path.realpath(diretorio), arquivo))

In [ ]:
def ingerir_dados(i):
    with open(i, 'r') as read_obj:
        try:
            rows = list(csv.reader(read_obj,delimiter=','))
            if rows[0] == cols.split(','): 
                rows.pop(0)
            cur.executemany("INSERT INTO matricula ("+cols+") VALUES ("+("?,"*102)+"?);",rows)
        except:
            return i

In [ ]:
for i in files:
   ingerir_dados(i)

In [ ]:
with open('../input/codigouf/UF.csv', 'r') as read_obj:
    rows = list(csv.reader(read_obj,delimiter=';'))
    for i in rows:
        cur.execute('UPDATE MATRICULA SET CO_UF= "'+i[1]+'" WHERE CO_UF="'+i[0]+'";')
        print(i[1]+' substituido por: '+i[0])

In [ ]:
with open('../input/codigocor/COR.csv', 'r') as read_obj:
    rows = list(csv.reader(read_obj,delimiter=';'))
    for i in rows:
        cur.execute('UPDATE MATRICULA SET TP_COR_RACA= "'+i[1]+'" WHERE TP_COR_RACA="'+i[0]+'";')
        print(i[1]+' substituido por: '+i[0])

In [ ]:
#Questão 1
cur.execute("SELECT CO_UF,TP_ETAPA_ENSINO,COUNT(*) FROM matricula GROUP BY CO_UF, TP_ETAPA_ENSINO HAVING TP_ETAPA_ENSINO='14' ORDER BY COUNT(*) DESC LIMIT 10")
q1 = cur.fetchall()
for i in q1:
    print(i)

In [ ]:
#Questão 2
cur.execute("SELECT CO_UF,TP_COR_RACA, COUNT(TP_COR_RACA) FROM matricula GROUP BY CO_UF,TP_COR_RACA")
q2 = cur.fetchall()
for i in q2:
    print(i)

In [ ]:
con.commit()
con.close()